In [52]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


In [53]:
import utils

In [54]:
itp_id = 300

In [55]:
pd.set_option("display.max_columns", 100)


In [56]:
analysis_date_start = '2022-05-01'
analysis_date_end = '2022-05-31'

In [57]:
rt_sched = query_sql(
    """
SELECT *
FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_sched_ran`
"""
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [58]:
rt_sched.head()

,calitp_itp_id,calitp_url_number,service_date,num_sched,num_rt,pct_ran
0,300,0,2022-06-12,770,533,0.69
1,300,0,2022-05-22,770,543,0.71
2,300,0,2022-05-15,771,527,0.68
3,300,0,2022-05-01,771,527,0.68
4,300,0,2022-06-26,772,500,0.65


In [59]:
rt_sched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   calitp_itp_id      61 non-null     int64  
 1   calitp_url_number  61 non-null     int64  
 2   service_date       61 non-null     object 
 3   num_sched          61 non-null     int64  
 4   num_rt             61 non-null     int64  
 5   pct_ran            61 non-null     float64
dtypes: float64(1), int64(4), object(1)
memory usage: 3.0+ KB


In [60]:
rt_sched['service_date'] = pd.to_datetime(rt_sched['service_date'])

In [61]:
rt_sched['weekday'] = pd.Series(rt_sched.service_date).dt.day_name()

## Using Functions (without DBT tables)

In [11]:
#gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [12]:
#gtfs_daily

In [13]:
#gtfs_daily.service_date.min()

In [14]:
#gtfs_daily.service_date.max()

In [15]:
#rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

In [16]:
#rt

In [17]:
#rt['str_len'] = rt.trip_id.str.len()

In [18]:
#rt.str_len.value_counts()

In [19]:
#len(rt)

In [20]:
#rt.trip_id.nunique()

In [21]:
#len(gtfs_daily)

In [22]:
#gtfs_daily.trip_id.nunique()

In [23]:
#date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [24]:

# from datetime import date, timedelta
# start_date = date(2022, 5, 1)
# end_date = date(2022, 5, 31)

In [25]:
# daterange = pd.date_range(start_date, end_date)

In [26]:
# (daterange)

In [27]:
#above range does not work with function

In [28]:
# dates = set()

# def daterange2(start, end):
#     for n in range(int((end - start).days) + 1):
#         yield start + timedelta(n)

# for single_date in daterange2(start_date, end_date):
#     dates.add(single_date.strftime('%Y-%m-%d'))

In [29]:
# (dates)

In [30]:
# rt['date'] = pd.to_datetime(rt['date'])

In [31]:
# gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [32]:
# single_date2 = '2022-05-08'

# (rt>>filter(_.date == single_date2))


In [33]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [34]:
# may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [35]:
# may

## Charting

In [62]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [63]:
rt_sched.sample()

,calitp_itp_id,calitp_url_number,service_date,num_sched,num_rt,pct_ran,weekday
49,300,0,2022-06-27,1465,1017,0.69,Monday


In [100]:
bar = (alt.Chart(rt_sched)
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=_dla_utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_ran', title=_dla_utils.labeling('pct_ran')),
            color=alt.Color(
                'pct_ran',
                scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                legend=alt.Legend(title=(_dla_utils.labeling('pct_ran')), symbolLimit=10)
            )
        ).properties(title='"Percent of Scheduled Trips with RT Data"'))
    
chart = styleguide.preset_chart_config(bar)
chart.properties(width=600)

alt.Chart(...)

In [95]:
bar = (alt.Chart(rt_sched)
        .mark_bar(size=9)
        .encode(
            x=alt.X('service_date', title=_dla_utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_ran', title=_dla_utils.labeling('pct_ran'))
        ).properties(title="Percent of Scheduled Trips with RT Data"))

## mean line
rule = alt.Chart(rt_sched).mark_rule(color='red').encode(
    y='mean(pct_ran):Q'
)


## rolling mean
# line = alt.Chart(rt_sched).mark_line(color='red').transform_window(
#     # The field to average
#     rolling_mean='mean(pct_ran)',
#     # The number of values before and after the current value to include.
#     frame=[-3, 0]
# ).encode(
#     x='service_date:O',
#     y='rolling_mean:Q'
# )

(bar + rule).properties(width=600)

alt.LayerChart(...)

In [65]:
df_long =  (rt_sched>>select(_.service_date,
                         _.num_sched,
                         _.num_rt
                        ) 
             >> gather('measure',
                       'value',
                       _.num_sched,
                       _.num_rt
                      )
            )

In [66]:
df_long

,service_date,measure,value
0,2022-06-12,num_sched,770
1,2022-05-22,num_sched,770
2,2022-05-15,num_sched,771
3,2022-05-01,num_sched,771
4,2022-06-26,num_sched,772
...,...,...,...
117,2022-05-16,num_rt,1030
118,2022-05-02,num_rt,1069
119,2022-06-06,num_rt,1048
120,2022-05-09,num_rt,1060


In [102]:
bar = (alt.Chart(df_long).mark_bar(opacity=1, size=9).encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('value', stack=None,  title=_dla_utils.labeling('number of trips')),
    color="measure")
      )

chart = styleguide.preset_chart_config(bar)
chart.properties(width=600)

alt.Chart(...)

In [72]:
rt_sched.sample()

,calitp_itp_id,calitp_url_number,service_date,num_sched,num_rt,pct_ran,weekday
44,300,0,2022-05-28,873,615,0.70,Saturday


In [43]:
week = rt_sched>>group_by(_.weekday)>>summarize(avg_pct_ran = _.pct_ran.mean())

In [44]:
week

,weekday,avg_pct_ran
0,Friday,0.91
1,Monday,0.73
2,Saturday,0.70
3,Sunday,0.69
4,Thursday,0.92
5,Tuesday,0.89
6,Wednesday,0.92


In [45]:
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week['weekday'] = pd.Categorical(week['weekday'], categories=cats, ordered=True)
week=week.sort_values('weekday')          

In [46]:
week

,weekday,avg_pct_ran
1,Monday,0.73
5,Tuesday,0.89
6,Wednesday,0.92
4,Thursday,0.92
0,Friday,0.91
2,Saturday,0.70
3,Sunday,0.69


In [47]:
bar = alt.Chart(week).mark_bar().encode(
    x=alt.X('weekday', title=_dla_utils.labeling('weekday'), sort=cats),
    y=alt.Y('avg_pct_ran', title=_dla_utils.labeling('avg_pct_ran')),
    color="weekday")
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [48]:
rt_sched>>filter(_.pct_ran>.95)

,calitp_itp_id,calitp_url_number,service_date,num_sched,num_rt,pct_ran,weekday
12,300,0,2022-06-23,1558,1481,0.95,Thursday
21,300,0,2022-05-30,829,800,0.97,Monday
51,300,0,2022-06-15,1505,1437,0.95,Wednesday
52,300,0,2022-06-16,1514,1487,0.98,Thursday
55,300,0,2022-06-17,1516,1507,0.99,Friday


In [107]:
rt_sched['month'] =  pd.Series(rt_sched.service_date).dt.month_name()

In [108]:
rt_sched>>group_by(_.month)>>summarize(avg_pct_ran = _.pct_ran.mean())

,month,avg_pct_ran
0,June,0.83
1,May,0.82


In [109]:
bar = alt.Chart((rt_sched>>group_by(_.month)>>summarize(avg_pct_ran = _.pct_ran.mean()))).mark_bar().encode(
    x=alt.X('month', title=_dla_utils.labeling('month'), sort=cats),
    y=alt.Y('avg_pct_ran', title=_dla_utils.labeling('avg_pct_ran')),
    color="month")
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)